## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carnarvon,AU,-24.8667,113.6333,71.67,64,40,16.11,scattered clouds
1,1,Cape Town,ZA,-33.9258,18.4232,54.27,92,75,2.30,broken clouds
2,2,Rikitea,PF,-23.1203,-134.9692,74.05,79,100,16.71,light rain
3,3,Iqaluit,CA,63.7506,-68.5145,26.33,92,20,10.00,few clouds
4,4,Bredasdorp,ZA,-34.5322,20.0403,45.72,87,97,4.61,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
16,16,Blackwater,AU,-23.5833,148.8833,83.32,26,0,2.01,clear sky
27,27,Butaritari,KI,3.0707,172.7902,83.73,73,11,10.29,few clouds
28,28,Thakurgaon,BD,26.0333,88.4667,86.68,68,6,4.23,clear sky
35,35,Soe,ID,-9.8607,124.2840,85.55,36,18,6.44,few clouds
39,39,Vaini,TO,-21.2000,-175.2000,75.36,73,20,12.66,few clouds
44,44,Nanakuli,US,21.3906,-158.1547,87.75,64,40,10.36,scattered clouds
45,45,Kapaa,US,22.0752,-159.3190,82.38,76,75,3.00,light rain
47,47,Sibolga,ID,1.7427,98.7792,85.33,67,100,2.86,overcast clouds
54,54,Umm Lajj,SA,25.0213,37.2685,81.10,64,11,12.68,few clouds
59,59,Kavieng,PG,-2.5744,150.7967,83.66,72,14,10.20,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                205
City                   205
Country                205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [6]:
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# Check if there are any empty cells with each column -- completed below function with "City" through "Current Description"
preferred_cities_df["Current Description"].eq("").sum()

0

In [8]:
preferred_cities_df.notnull().sum()

City ID                205
City                   205
Country                205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# As indicated above, there are no empty rows or empty values in the dataframe. I conducted multiple checks to ensure this.

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Blackwater,AU,83.32,clear sky,-23.5833,148.8833,
27,Butaritari,KI,83.73,few clouds,3.0707,172.7902,
28,Thakurgaon,BD,86.68,clear sky,26.0333,88.4667,
35,Soe,ID,85.55,few clouds,-9.8607,124.2840,
39,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
44,Nanakuli,US,87.75,scattered clouds,21.3906,-158.1547,
45,Kapaa,US,82.38,light rain,22.0752,-159.3190,
47,Sibolga,ID,85.33,overcast clouds,1.7427,98.7792,
54,Umm Lajj,SA,81.10,few clouds,25.0213,37.2685,
59,Kavieng,PG,83.66,light rain,-2.5744,150.7967,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat}, {lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Blackwater,AU,83.32,clear sky,-23.5833,148.8833,Blackwater Motor Inn
27,Butaritari,KI,83.73,few clouds,3.0707,172.7902,Isles Sunset Lodge
28,Thakurgaon,BD,86.68,clear sky,26.0333,88.4667,Chowdhury Khokon Fisheries Head Office
35,Soe,ID,85.55,few clouds,-9.8607,124.2840,Dena Hotel
39,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
44,Nanakuli,US,87.75,scattered clouds,21.3906,-158.1547,Camp Pālehua
45,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
47,Sibolga,ID,85.33,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
54,Umm Lajj,SA,81.10,few clouds,25.0213,37.2685,Moon light Furnished Units
59,Kavieng,PG,83.66,light rain,-2.5744,150.7967,Nusa Island Retreat


In [13]:
# Check if there are any missing values in Hotel Name
hotel_df["Hotel Name"].eq("").sum()

12

In [14]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df.loc[(hotel_df["Hotel Name"] == "")]=np.nan
hotel_df.count()


City                   193
Country                193
Max Temp               193
Current Description    193
Lat                    193
Lng                    193
Hotel Name             193
dtype: int64

In [15]:
clean_hotel_df = hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng", "Hotel Name"]].copy()
clean_hotel_df.count()

City                   193
Country                193
Max Temp               193
Current Description    193
Lat                    193
Lng                    193
Hotel Name             193
dtype: int64

In [16]:
clean_hotel_df.dropna(inplace=True)

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "..\Vacation_Search\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} F</dd>
<dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))